In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# memory
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import ChatMessage
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

# pinecone

In [3]:
# data load and split
loaders = [
    PDFPlumberLoader("./data/RAG/Adaptive_RAG.pdf"),
    PDFPlumberLoader("./data/RAG/Naive_RAG.pdf"),
    PDFPlumberLoader("./data/RAG/RAPTOR_RAG.pdf"),
    PDFPlumberLoader("./data/RAG/Self_RAG.pdf")
]

docs = []

for loader in loaders:
    docs.extend(loader.load())


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

len(documents)

341

In [4]:
vars(documents[0])

{'id': None,
 'metadata': {'source': './data/RAG/Adaptive_RAG.pdf',
  'file_path': './data/RAG/Adaptive_RAG.pdf',
  'page': 0,
  'total_pages': 15,
  'Author': '',
  'CreationDate': 'D:20240329003444Z',
  'Creator': 'LaTeX with hyperref',
  'Keywords': '',
  'ModDate': 'D:20240329003444Z',
  'PTEX.Fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
  'Producer': 'pdfTeX-1.40.25',
  'Subject': '',
  'Title': '',
  'Trapped': 'False'},
 'page_content': 'Adaptive-RAG: Learning to Adapt Retrieval-Augmented\nLarge Language Models through Question Complexity\nSoyeongJeong1 JinheonBaek2 SukminCho1 SungJuHwang1,2 JongC.Park1*\nSchoolofComputing1 GraduateSchoolofAI2\nKoreaAdvancedInstituteofScienceandTechnology1,2\n{starsuzi,jinheon.baek,nelllpic,sjhwang82,jongpark}@kaist.ac.kr\nAbstract\n51\nRetrieval-AugmentedLargeLanguageModels 50\n(LLMs),whichincorporatethenon-parametric 49\nknowledgefromexternalknowledgebasesinto 48\nLLMs,haveemergedasapro

In [5]:
documents[0].metadata

{'source': './data/RAG/Adaptive_RAG.pdf',
 'file_path': './data/RAG/Adaptive_RAG.pdf',
 'page': 0,
 'total_pages': 15,
 'Author': '',
 'CreationDate': 'D:20240329003444Z',
 'Creator': 'LaTeX with hyperref',
 'Keywords': '',
 'ModDate': 'D:20240329003444Z',
 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'Producer': 'pdfTeX-1.40.25',
 'Subject': '',
 'Title': '',
 'Trapped': 'False'}

파인콘
preprocess - sparse encoder생성(kiwi tokenizer, stopwords(불용어)) - 인덱스 생성 또는 불러오기 - DB에 데이터 추가 - init pinecone - kiwihybridretriever 생성


In [6]:
# preprocessing documents for pinecone
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs = documents,
    metadata_keys = ["source", "page"],
    min_length=5,
    use_basename=True
)

  0%|          | 0/341 [00:00<?, ?it/s]

In [7]:
print(len(contents))
print(len(metadatas))

341
2


In [8]:
metadatas.keys()

dict_keys(['source', 'page'])

In [9]:
contents[:5]

['Adaptive-RAG: Learning to Adapt Retrieval-Augmented\nLarge Language Models through Question Complexity\nSoyeongJeong1 JinheonBaek2 SukminCho1 SungJuHwang1,2 JongC.Park1*\nSchoolofComputing1 GraduateSchoolofAI2\nKoreaAdvancedInstituteofScienceandTechnology1,2\n{starsuzi,jinheon.baek,nelllpic,sjhwang82,jongpark}@kaist.ac.kr\nAbstract\n51\nRetrieval-AugmentedLargeLanguageModels 50\n(LLMs),whichincorporatethenon-parametric 49\nknowledgefromexternalknowledgebasesinto 48\nLLMs,haveemergedasapromisingapproach 47\ntoenhancingresponseaccuracyinseveraltasks, 0.5 1.0 1.5 2.0 2.5 3.0 3.5\nTime per Query\nsuchasQuestion-Answering(QA).However,\neventhoughtherearevariousapproachesdeal-\ningwithqueriesofdifferentcomplexities,they\neitherhandlesimplequerieswithunnecessary\ncomputational overhead or fail to adequately\naddress complex multi-step queries; yet, not\nalluserrequestsfallintoonlyoneofthesim-\nple or complex categories. In this work, we\nproposeanoveladaptiveQAframeworkthat',
 'proposeanove

In [10]:
len(contents), len(metadatas["source"]), len(metadatas['page'])

(341, 341, 341)

In [11]:
# # pinecone : Sparse Encoder생성 - KiwiTokenizer + 불용어(stopwords)
# from langchain_teddynote.korean import stopwords
# from langchain_teddynote.community.pinecone import create_sparse_encoder, fit_sparse_encoder

# stopword = stopwords()
# stopword

# sparse_encoder = create_sparse_encoder(stopword, mode='kiwi')

# save_path = fit_sparse_encoder(
#     sparse_encoder=sparse_encoder, contents=contents, save_path="./sparse_encoder.pkl"
# )
# # 저장한 sparse encoder불러오기
# # from langchain_teddynote.community.pinecone import load_sparse_encoder

# # sparse_encoder = load_sparse_encoder("./sparse_encoder.pkl")

  0%|          | 0/341 [00:00<?, ?it/s]

[fit_sparse_encoder]
Saved Sparse Encoder to: ./sparse_encoder.pkl


In [12]:
# # pinecone : create index
# import os
# from langchain_teddynote.community.pinecone import create_index
# pc_index = create_index(
#     api_key=os.environ["PINECONE_API_KEY"],
#     index_name="jaeho-note-index", # 인덱스 이름을 지정합니다.
#     dimension=1536, # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
#     metric="dotproduct" # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine) Hybrid search를 위해선 dotproduct를 사용
# )

[create_index]
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 1458},
                'jaehonote-namespace-0': {'vector_count': 619},
                'jaehonote-namespace-2': {'vector_count': 341}},
 'total_vector_count': 2418}


In [13]:
# Pinecone: DB index에추가(Upsert)
# import os

# from langchain_teddynote.community.pinecone import upsert_documents_parallel
# openai_embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# upsert_documents_parallel(
#     index=pc_index,
#     namespace="jaehonote-namespace-2",
#     contents=contents,
#     metadatas=metadatas,
#     sparse_encoder=sparse_encoder,
#     embedder=openai_embedding,
#     batch_size=32,
#     max_workers=30
# )
# 2077+341

In [11]:
import os
from langchain_teddynote.korean import stopwords
stopword = stopwords()
# pinecone index
from langchain_teddynote.community.pinecone import init_pinecone_index
openai_embedding = OpenAIEmbeddings(model='text-embedding-3-small')
pinecone_params = init_pinecone_index(
    index_name="jaeho-note-index",
    namespace="jaehonote-namespace-2",
    api_key=os.environ["PINECONE_API_KEY"],
    sparse_encoder_path="./sparse_encoder.pkl",
    stopwords=stopword,
    tokenizer="kiwi",
    embeddings=openai_embedding,
    top_k=5,
    alpha=0.5
)


[init_pinecone_index]
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 1458},
                'jaehonote-namespace-0': {'vector_count': 619},
                'jaehonote-namespace-2': {'vector_count': 341}},
 'total_vector_count': 2418}


In [12]:
# pinecone kiwi hybridretriever
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever

pinecone_retriever = PineconeKiwiHybridRetriever(**pinecone_params)

In [13]:
pinecone_retriever.invoke("self-rag에 대해서 알려줘")

[Document(metadata={'page': 9.0, 'source': 'Self_RAG.pdf', 'score': 0.2701767}, page_content='outputissupportedbythecitedevidence). Humanannotatorsfind SELF-RAG answersareoften\nplausibleandsupportedbyrelevantpassageswithhigherS&Pscoresonshort-formPopQA,whichis\nconsistentwithMenicketal.(2022). Humanannotatorsalsofind ISREL and ISSUP reflectiontoken\npredictionsaremostlyalignedwiththeirassessments. AppendixTable6showsseveralannotated\nexamplesandexplanationsonassessments.\n6 CONCLUSION\nThisworkintroducesSELF-RAG,anewframeworktoenhancethequalityandfactualityofLLMs\nthroughretrievalondemandandself-reflection. SELF-RAGtrainsanLMtolearntoretrieve,generate,\nand critique text passages and its own generation by predicting the next tokens from its original\nvocabularyaswellasnewlyaddedspecialtokens,calledreflectiontokens. SELF-RAGfurtherenables\nthetailoringofLMbehaviorsattesttimebyleveragingreflectiontokens. Ourholisticevaluationson\nsixtasksusingmultiplemetricsdemonstratethatSELF-RAGsignif

In [14]:
response = pinecone_retriever.invoke("self-rag에 대해서 알려줘.")
for result in response:
    print(result.page_content)
    print(result.metadata)
    print("\n\n")

outputissupportedbythecitedevidence). Humanannotatorsfind SELF-RAG answersareoften
plausibleandsupportedbyrelevantpassageswithhigherS&Pscoresonshort-formPopQA,whichis
consistentwithMenicketal.(2022). Humanannotatorsalsofind ISREL and ISSUP reflectiontoken
predictionsaremostlyalignedwiththeirassessments. AppendixTable6showsseveralannotated
examplesandexplanationsonassessments.
6 CONCLUSION
ThisworkintroducesSELF-RAG,anewframeworktoenhancethequalityandfactualityofLLMs
throughretrievalondemandandself-reflection. SELF-RAGtrainsanLMtolearntoretrieve,generate,
and critique text passages and its own generation by predicting the next tokens from its original
vocabularyaswellasnewlyaddedspecialtokens,calledreflectiontokens. SELF-RAGfurtherenables
thetailoringofLMbehaviorsattesttimebyleveragingreflectiontokens. Ourholisticevaluationson
sixtasksusingmultiplemetricsdemonstratethatSELF-RAGsignificantlyoutperformsLLMswith
moreparametersorwithconventionalretrieval-augmentedgenerationapproaches.
10
{'

In [15]:
# multi-query
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever = pinecone_retriever,
    llm=llm
)

In [16]:
# 멀티쿼리가 정확도가 떨어질수도 있다.
response = multiquery_retriever.invoke("self-rag에 대해서 알려줘.")
for result in response:
    print(result.page_content)
    print(result.metadata)
    print("\n\n")

outputissupportedbythecitedevidence). Humanannotatorsfind SELF-RAG answersareoften
plausibleandsupportedbyrelevantpassageswithhigherS&Pscoresonshort-formPopQA,whichis
consistentwithMenicketal.(2022). Humanannotatorsalsofind ISREL and ISSUP reflectiontoken
predictionsaremostlyalignedwiththeirassessments. AppendixTable6showsseveralannotated
examplesandexplanationsonassessments.
6 CONCLUSION
ThisworkintroducesSELF-RAG,anewframeworktoenhancethequalityandfactualityofLLMs
throughretrievalondemandandself-reflection. SELF-RAGtrainsanLMtolearntoretrieve,generate,
and critique text passages and its own generation by predicting the next tokens from its original
vocabularyaswellasnewlyaddedspecialtokens,calledreflectiontokens. SELF-RAGfurtherenables
thetailoringofLMbehaviorsattesttimebyleveragingreflectiontokens. Ourholisticevaluationson
sixtasksusingmultiplemetricsdemonstratethatSELF-RAGsignificantlyoutperformsLLMswith
moreparametersorwithconventionalretrieval-augmentedgenerationapproaches.
10
{'

In [17]:
# reranker

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개모델 선택
compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=multiquery_retriever
)

In [18]:
# Reoder : retirever에 이것을 덧댄다.
from langchain_community.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda

def reorder_documents(docs):
    #재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    return reordered_docs

In [19]:
from langchain_core.prompts import PromptTemplate
# prompt
template = """
    당신은 주어진 문서에 대해서 QA를 해주는 assistant bot입니다.
    주어진 문서를 이용해서 답변해 주세요
    만약 주어진 질문에 대해서 모른다면, 모른다고 답변해 주세요
    문서의 출처와 페이지 넘버를 작성해주세요
    한국어로 답변해주세요

    #Example Format:
    (brief summary of the answer)
    (table)
    (detailed answer to the question)

    **출처**
    - (page source and page number)

    # Question:
    {question}

    # Context:
    {context}

    # Answer
"""
prompt = PromptTemplate.from_template(template, input_variable=["question", "context"])

In [20]:
# chain
chain = ({"context":compression_retriever| RunnableLambda(reorder_documents), "question": RunnablePassthrough()}
         |prompt
         |llm
         |StrOutputParser()
         )

In [21]:
print(chain.invoke("AdaptiveRAG에 대해서 설명해 주세요"))

Adaptive-RAG는 다양한 복잡성을 가진 쿼리를 처리하기 위해 설계된 Retrieval-Augmented Generation 프레임워크입니다. 이 시스템은 쿼리의 복잡성에 따라 동적으로 쿼리 처리 전략을 조정하여, 가장 간단한 쿼리부터 단일 단계 접근 방식까지 다양한 쿼리를 효과적으로 처리할 수 있습니다.

| Adaptive-RAG의 특징 | 설명 |
|---------------------|------|
| 동적 조정          | 쿼리의 복잡성에 따라 처리 전략을 조정 |
| 다양한 쿼리 처리   | 간단한 쿼리부터 복잡한 쿼리까지 지원 |
| 효율성              | 기존 방법보다 더 효과적이고 효율적임 |

Adaptive-RAG는 쿼리 복잡성을 자동으로 분류하고, 모델 예측 결과에 기반하여 새로운 데이터를 생성하는 기능을 가지고 있습니다. 그러나 쿼리 복잡성을 잘못 라벨링할 가능성도 있으며, 향후 연구에서는 다양한 쿼리 복잡성을 주석 처리한 새로운 데이터셋을 생성할 필요가 있습니다.

**출처**
- Adaptive_RAG.pdf, 페이지 8, 13


In [22]:
print(chain.invoke("SelfRAG에 대해서 설명해 주세요"))

SelfRAG는 대형 언어 모델(LLM)의 품질과 사실성을 향상시키기 위해 설계된 새로운 프레임워크입니다. 이 프레임워크는 Self-Reflective Retrieval-Augmented Generation의 약자로, LLM이 관련 지식을 검색하고 생성하며 비판하는 과정을 통해 학습하도록 돕습니다. 기존의 Retrieval-Augmented Generation(RAG) 접근 방식은 무작위로 고정된 수의 검색된 구문을 통합하는 방식으로, 필요하지 않거나 관련성이 없는 구문을 포함할 수 있어 LLM의 다양성을 저하시킬 수 있습니다. SelfRAG는 이러한 문제를 해결하기 위해 LLM의 응답 품질을 높이고 사실성을 강화하는 데 중점을 둡니다.

| 항목               | 내용                                                                 |
|------------------|----------------------------------------------------------------------|
| 프레임워크 이름      | Self-Reflective Retrieval-Augmented Generation (SelfRAG)            |
| 목적               | LLM의 품질과 사실성 향상                                             |
| 문제점            | 기존 RAG의 무작위 검색 및 통합 방식으로 인한 LLM의 다양성 저하       |
| 해결책            | LLM이 검색, 생성, 비판하는 과정을 통해 학습하도록 지원              |

SelfRAG는 LLM이 더 정확하고 유용한 응답을 생성할 수 있도록 돕는 혁신적인 접근 방식입니다.

**출처**
- Self_RAG.pdf, 페이지 0


In [23]:
print(chain.invoke("SelfRAG와 AdaptiveRAG로 무엇을 할 수 있는지 설명해 주세요"))

SelfRAG와 AdaptiveRAG는 각각의 방식으로 정보 검색 및 생성 작업을 개선하는 기술입니다.

| 기술         | 기능 설명                                                                                     |
|--------------|----------------------------------------------------------------------------------------------|
| SelfRAG      | 모델이 생성한 각 세그먼트 후 자신의 예측을 평가하고, 사용자 선호에 맞춰 검색 빈도를 조정할 수 있는 커스터마이징 가능한 디코딩 알고리즘을 제공합니다. |
| AdaptiveRAG  | 다양한 사용자 쿼리에 대한 실용적인 시나리오에서의 적용 가능성을 검증하며, 비 offensive한 입력을 고려하여 적절한 응답을 생성하는 데 중점을 둡니다. |

SelfRAG는 각 생성 세그먼트 후 모델이 자신의 예측을 평가하는 기능을 포함하고 있으며, 이를 통해 사용자 선호에 맞춰 모델의 행동을 조정할 수 있습니다. AdaptiveRAG는 다양한 사용자 쿼리에 대한 실험 결과를 통해 실제 시나리오에서의 적용 가능성을 입증하고, 비 offensive한 입력을 처리하는 데 중점을 두고 있습니다.

**출처**
- Self_RAG.pdf, 페이지 1
- Adaptive_RAG.pdf, 페이지 9
